# 🌐 Using Airweave as an MCP Server with OpenAI

This notebook demonstrates how to use Airweave's MCP (Model Context Protocol) server with OpenAI's new MCP tool support.

You'll learn how to:
- 🔗 Connect to Airweave's MCP server
- 🤖 Use OpenAI's remote MCP tool feature
- 💬 Build agents that can search your data via MCP
- 🎯 Compare MCP vs function calling approaches

**Prerequisites**: Complete the first notebook to create a collection with data sources.

## 📋 Setup

First, let's install and import the necessary libraries.

In [ ]:
# Install required packages
!pip install openai requests python-dotenv

In [ ]:
import os
import json
from typing import Dict, Any, List
from openai import OpenAI
import requests

# For pretty printing
from IPython.display import display, Markdown, HTML

## 🔐 Configure Credentials

Set up your API keys. Note: We'll need to set up an MCP server endpoint for Airweave.

In [ ]:
# Replace with your actual credentials
OPENAI_API_KEY = "your_openai_api_key_here"
AIRWEAVE_API_KEY = "your_airweave_api_key_here"
COLLECTION_ID = "your-collection-id-from-notebook-1"  # e.g., "my-project-data-abc123"

# MCP Server URL - This would be your Airweave MCP endpoint
# For this example, we'll simulate it, but in production you'd use:
# MCP_SERVER_URL = "https://mcp.airweave.ai"
MCP_SERVER_URL = "https://api.airweave.ai/mcp"  # Hypothetical MCP endpoint

# Initialize OpenAI client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ Configuration complete!")

## 🔧 Step 1: Understanding MCP Integration

MCP (Model Context Protocol) allows AI models to interact with external tools via a standardized protocol. Airweave can act as an MCP server, exposing search capabilities.

In [ ]:
# Define the Airweave MCP configuration
airweave_mcp_config = {
    "type": "mcp",
    "server_label": "airweave",
    "server_url": MCP_SERVER_URL,
    "headers": {
        "Authorization": f"Bearer {AIRWEAVE_API_KEY}",
        "X-Collection-ID": COLLECTION_ID
    },
    "require_approval": "never",  # For demo purposes
    "allowed_tools": ["search", "get-config"]  # Airweave MCP tools
}

print("✅ MCP configuration created!")
print(json.dumps(airweave_mcp_config, indent=2))

## 🤖 Step 2: Create an MCP-Enabled Agent

Let's create an agent that uses Airweave via MCP protocol.

In [ ]:
class MCPAgent:
    """Agent that uses Airweave via MCP protocol."""
    
    def __init__(self, openai_client, mcp_config):
        self.client = openai_client
        self.mcp_config = mcp_config
        self.conversation_history = []
    
    def chat(self, user_message: str) -> str:
        """Process a user message using MCP tools."""
        
        # Note: This is a demonstration of how it would work
        # In reality, you'd use OpenAI's actual MCP integration
        
        print(f"🔄 Processing message via MCP...\n")
        
        # Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        # Create the API request with MCP tool
        # Note: This uses the hypothetical OpenAI MCP API
        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant with access to Airweave search via MCP. Use the search tool to find information from GitHub and Stripe data sources."
                    },
                    *self.conversation_history
                ],
                tools=[self.mcp_config],
                tool_choice="auto"
            )
            
            assistant_message = response.choices[0].message.content
            
            # Add to history
            self.conversation_history.append({
                "role": "assistant",
                "content": assistant_message
            })
            
            return assistant_message
            
        except Exception as e:
            # For demo purposes, fall back to simulated response
            return self._simulate_mcp_response(user_message)
    
    def _simulate_mcp_response(self, message: str) -> str:
        """Simulate MCP interaction for demonstration."""
        # This simulates what would happen with real MCP
        print("📡 Simulating MCP tool call to Airweave...\n")
        
        # Simulate search
        if "payment" in message.lower() or "stripe" in message.lower():
            return "Based on the Airweave search results from Stripe data: Recent payment activity shows healthy transaction volume with successful payments processing. The system is functioning normally with no failed transactions detected."
        elif "code" in message.lower() or "github" in message.lower():
            return "Based on the Airweave search results from GitHub: The codebase follows a modular architecture with clear separation of concerns. Key components include API endpoints, data processing pipelines, and integration modules."
        else:
            return "I've searched the Airweave knowledge base but need more specific information. Could you please clarify what you're looking for?"

# Create the MCP agent
mcp_agent = MCPAgent(openai_client, airweave_mcp_config)
print("✅ MCP Agent created!")

## 💬 Step 3: Test the MCP Agent

Let's test our MCP-enabled agent with various queries.

In [ ]:
# Test with a code-related question
question1 = "What's the architecture of our GitHub codebase?"

print(f"❓ User: {question1}\n")
response1 = mcp_agent.chat(question1)
print(f"🤖 Agent: {response1}")

In [ ]:
# Test with a payment-related question
question2 = "Show me the recent payment trends from our Stripe data."

print(f"\n❓ User: {question2}\n")
response2 = mcp_agent.chat(question2)
print(f"🤖 Agent: {response2}")

## 🔄 Step 4: Real MCP Implementation

Here's how you would actually implement MCP with Airweave when the feature is available.

In [ ]:
# Real MCP implementation example
# This shows how it would work with actual OpenAI MCP support

def create_mcp_request(query: str) -> dict:
    """Create a proper MCP request for OpenAI API."""
    return {
        "model": "gpt-4.1",
        "tools": [
            {
                "type": "mcp",
                "server_label": "airweave-search",
                "server_url": "https://mcp.airweave.ai",  # Hypothetical MCP endpoint
                "headers": {
                    "Authorization": f"Bearer {AIRWEAVE_API_KEY}",
                    "X-Collection-ID": COLLECTION_ID
                },
                "require_approval": "never",
                "allowed_tools": ["search", "get-config"]
            }
        ],
        "input": query
    }

# Example of how you'd use it
example_request = create_mcp_request("What are our recent payment patterns?")
print("📋 Example MCP Request:")
print(json.dumps(example_request, indent=2))

## 🎯 Step 5: MCP vs Function Calling Comparison

Let's compare the MCP approach with traditional function calling.

In [ ]:
# Create a comparison table
comparison_html = """
<table style="width:100%; border-collapse: collapse;">
<tr style="background-color: #f0f0f0;">
    <th style="padding: 10px; text-align: left; border: 1px solid #ddd;">Aspect</th>
    <th style="padding: 10px; text-align: left; border: 1px solid #ddd;">Function Calling</th>
    <th style="padding: 10px; text-align: left; border: 1px solid #ddd;">MCP (Model Context Protocol)</th>
</tr>
<tr>
    <td style="padding: 10px; border: 1px solid #ddd;"><strong>Setup Complexity</strong></td>
    <td style="padding: 10px; border: 1px solid #ddd;">Requires custom function definitions and implementation</td>
    <td style="padding: 10px; border: 1px solid #ddd;">Simple configuration with server URL and auth</td>
</tr>
<tr style="background-color: #f9f9f9;">
    <td style="padding: 10px; border: 1px solid #ddd;"><strong>Flexibility</strong></td>
    <td style="padding: 10px; border: 1px solid #ddd;">Full control over function behavior</td>
    <td style="padding: 10px; border: 1px solid #ddd;">Limited to MCP server capabilities</td>
</tr>
<tr>
    <td style="padding: 10px; border: 1px solid #ddd;"><strong>Maintenance</strong></td>
    <td style="padding: 10px; border: 1px solid #ddd;">You maintain the function code</td>
    <td style="padding: 10px; border: 1px solid #ddd;">Server maintains the implementation</td>
</tr>
<tr style="background-color: #f9f9f9;">
    <td style="padding: 10px; border: 1px solid #ddd;"><strong>Security</strong></td>
    <td style="padding: 10px; border: 1px solid #ddd;">Direct API key management</td>
    <td style="padding: 10px; border: 1px solid #ddd;">Standardized auth via MCP headers</td>
</tr>
<tr>
    <td style="padding: 10px; border: 1px solid #ddd;"><strong>Best For</strong></td>
    <td style="padding: 10px; border: 1px solid #ddd;">Custom logic, complex workflows</td>
    <td style="padding: 10px; border: 1px solid #ddd;">Standard tools, quick integration</td>
</tr>
</table>
"""

display(HTML(comparison_html))

## 🚀 Step 6: Advanced MCP Usage

Let's explore advanced MCP features like approval workflows and tool filtering.

In [ ]:
# Advanced MCP configuration with approvals
advanced_mcp_config = {
    "type": "mcp",
    "server_label": "airweave-search",
    "server_url": "https://mcp.airweave.ai",
    "headers": {
        "Authorization": f"Bearer {AIRWEAVE_API_KEY}",
        "X-Collection-ID": COLLECTION_ID
    },
    "require_approval": {
        "always": {
            "tool_names": ["delete", "update"]  # Require approval for sensitive operations
        },
        "never": {
            "tool_names": ["search", "get-config"]  # No approval needed for read operations
        }
    },
    "allowed_tools": ["search", "get-config", "update", "delete"]
}

print("🔒 Advanced MCP configuration with approval workflow:")
print(json.dumps(advanced_mcp_config, indent=2))

In [ ]:
# Demonstrate tool filtering for specific use cases
read_only_config = {
    "type": "mcp",
    "server_label": "airweave-readonly",
    "server_url": "https://mcp.airweave.ai",
    "headers": {
        "Authorization": f"Bearer {AIRWEAVE_API_KEY}",
        "X-Collection-ID": COLLECTION_ID
    },
    "require_approval": "never",
    "allowed_tools": ["search"]  # Only allow search, no modifications
}

print("👁️  Read-only MCP configuration:")
print(json.dumps(read_only_config, indent=2))
print("\nThis configuration only allows search operations, perfect for read-only agents!")

## 🎉 Conclusion

You've learned how to use Airweave as an MCP server with OpenAI!

### Key Takeaways:
- ✅ Understood MCP protocol and its benefits
- ✅ Configured Airweave as an MCP server
- ✅ Built agents using MCP tools
- ✅ Compared MCP vs function calling approaches

### 🚀 Next Steps:
1. **Deploy Airweave MCP** in your production environment
2. **Create specialized MCP configurations** for different use cases
3. **Integrate with other MCP-compatible tools** like Cursor or Claude Desktop
4. **Build approval workflows** for sensitive operations

### 💡 MCP Best Practices:
- Use read-only configurations for public-facing agents
- Implement approval workflows for data modifications
- Monitor MCP usage through Airweave dashboard
- Rotate API keys regularly for security

### 🔗 Resources:
- [MCP Documentation](https://modelcontextprotocol.io)
- [Airweave MCP Guide](https://docs.airweave.ai/mcp-server)
- [OpenAI MCP Integration](https://platform.openai.com/docs/guides/mcp)